In [4]:
import gzip
import random
import string
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wingfungleung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def parse(f):
    for l in gzip.open(f):
        yield eval(l)
dataset = list(parse('train_Category.json.gz'))

In [6]:
def readJSON(path):
    for l in gzip.open(path, 'rt'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['genreID']
        except Exception as e:
            g = None
        yield u,g,d
dataset

[{'userID': 'u74382925',
  'genre': 'Adventure',
  'early_access': False,
  'reviewID': 'r75487422',
  'hours': 4.1,
  'text': 'Short Review:\nA good starting chapter for this series, despite the main character being annoying (for now) and a short length. The story is good and actually gets more interesting. Worth the try.\nLong Review:\nBlackwell Legacy is the first on the series of (supposedly) 5 games that talks about the main protagonist, Rosangela Blackwell, as being a so called Medium, and in this first chapter we get to know how her story will start and how she will meet her adventure companion Joey...and really, that\'s really all for for now and that\'s not a bad thing, because in a way this game wants to show how hard her new job is, and that she cannot escape her destiny as a medium.\nMy biggest complain for this chapter, except the short length, it\'s the main protagonist being a "bit" too annoying to be likeable, and most of her dialogues will always be about complaining o

In [7]:
#shuffling data
random.shuffle(dataset)

In [8]:
#split data
train_set, validation_set = train_test_split(dataset, test_size=0.05714, random_state=42)

In [9]:
#Unique words in the train set
stop_words = set(stopwords.words('english')) 
wordCount = defaultdict(int)
for d in train_set:
    for w in d['text'].split():
        if w not in stop_words:
            wordCount[w] += 1


### Ignore capitalization and remove punctuation in the train set and stemming

wordCount = defaultdict(int)
punctuation = set(string.punctuation)

for d in train_set:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

        
### Just take the most popular words in train set
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
common_1000_words = [(x[0], x[1]) for x in counts[:1000]]
common_10_words = [(x[0], x[1]) for x in counts[:10]]



###create bag of words as X and genreID labels as y for train set
words = [x[1] for x in counts[:6000]] #first 1000 common words
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
X = []
bag_of_words = [0 for y in range(6000)]
train_hours = [ train_set[i]['hours'] for i in range(len(train_set))]
count = 0
for d in train_set:
    for w in d['text'].split():
        if w in wordSet:
            bag_of_words[wordId[w]] += 1
    bag_of_words.append(train_hours[count])
    X.append(bag_of_words)
    count += 1
    bag_of_words = [0 for y in range(6000)]
####


y = [d['genreID'] for d in train_set]
len(wordCount)

154718

In [31]:
#Unique words in the validation set
stop_words = set(stopwords.words('english')) 
wordCount2 = defaultdict(int)
for d in validation_set:
    for w in d['text'].split():
        if w not in stop_words:
            wordCount2[w] += 1


### Ignore capitalization and remove punctuation for validation set

wordCount2 = defaultdict(int)
punctuation2 = set(string.punctuation)

for d in validation_set:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation2])
    for w in r.split():
        wordCount2[w] += 1

        
### Just take the most popular words in validation set
val_counts = [(wordCount2[w], w) for w in wordCount2]
val_counts.sort()
val_counts.reverse()
val_common_1000_words = [(x[0], x[1]) for x in val_counts[:1000]]
val_common_10_words = [(x[0], x[1]) for x in val_counts[:10]]


###create bag of words as X_val and  genreID labels as y_val for validation set
words = [x[1] for x in val_counts[:6000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
X_val = []
bag_of_words = [0 for y in range(6000)]
val_hours = [ validation_set[i]['hours'] for i in range(len(validation_set))]
count = 0
for d in validation_set:
    for w in d['text'].split():
        if w in wordSet:
            bag_of_words[wordId[w]] += 1
    bag_of_words.append(val_hours[count])
    X_val.append(bag_of_words)
    count += 1
    bag_of_words = [0 for y in range(6000)]

y_val = [d['genreID'] for d in validation_set]

In [32]:
#train model
from scipy.sparse import lil_matrix
X_lil = lil_matrix(X)
clf = LogisticRegression(random_state=0, solver='lbfgs', fit_intercept=True, max_iter=10000,\
                         C=10).fit(X_lil, y)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
#predict
predict_output = clf.predict(X_val)
predict_output

array([0, 1, 1, ..., 0, 0, 0])

In [34]:
#accuracy
count = 0
for i in range(len(predict_output)):
    if predict_output[i] == y_val[i]:
        count += 1
accuracy = count / len(predict_output)
accuracy

0.5001

In [35]:
###predict output using test set
### Category prediction baseline: Just consider some of the most common words from each category
catDict = {
  "Action": 0,
  "Strategy": 1,
  "RPG": 2,
  "Adventure": 3,
  "Sport": 4
}

a = []
X = []
y = []
for u,_,data in readJSON("test_Category.json.gz"):
    words = [x[1] for x in counts[:6000]]
    wordId = dict(zip(words, range(len(words))))
    wordSet = set(words)
    bag_of_words = [0 for y in range(6000)]
    for w in data['text'].split():
        if w in wordSet:
            bag_of_words[wordId[w]] += 1
    bag_of_words.append(data['hours'])
    X.append(bag_of_words)
    
    
predict_output = clf.predict(X)  

In [36]:
predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
count = 0
for l in open("pairs_Category.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,r = l.strip().split('-')
    predictions.write(u + '-' + r + "," + str(predict_output[count]) + "\n")
    count += 1
predictions.close()